# HT 2 : Сжатие модели DistilBERT
---
Задача: уменьшить размер модели в 2 раза с минимальной потерей качества

### 🔧 Подготовка среды

In [4]:
# !pip3 install torch torchvision transformers

### 1️⃣ Импорт библиотек и загрузка модели

In [2]:

from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from datasets import load_dataset
import torch, os

MODEL_ID = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID)

dataset = load_dataset("imdb", split="test", )


### 2️⃣ Проверка работы исходной модели

In [3]:
clf = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

texts = [
    "I loved this movie, it was fantastic!",
    "This was a terrible waste of time.",
    "Not bad, but could be better.",
]

for t in texts:
    print(f"\nText: {t}")
    print("Prediction:", clf(t))


Device set to use cpu



Text: I loved this movie, it was fantastic!
Prediction: [{'label': 'POSITIVE', 'score': 0.9998804330825806}]

Text: This was a terrible waste of time.
Prediction: [{'label': 'NEGATIVE', 'score': 0.9998123049736023}]

Text: Not bad, but could be better.
Prediction: [{'label': 'NEGATIVE', 'score': 0.8015893697738647}]


### 3️⃣ Замер размера исходной модели

In [4]:

torch.save(model.state_dict(), "model_fp32.pt")
size_fp32 = os.path.getsize("model_fp32.pt") / 1e6
print(f" Размер исходной модели: {size_fp32:.1f} MB")


 Размер исходной модели: 267.9 MB


### 4️⃣ Квантование

In [5]:
from hw2 import *

In [6]:
quantized_model = quantize_model(model, modules_to_quantize = {nn.Linear})
size_quantized = save_model(quantized_model, "model_quantized.pt")

print(f"Размер квантованной модели: {size_quantized:.1f} MB")
print(f"Уменьшение размера: {(1 - size_quantized/size_fp32)*100:.1f}%")

Размер квантованной модели: 138.7 MB
Уменьшение размера: 48.2%


### 5️⃣ Оценка качества

In [28]:
print("Evaluating original model...")
original_metrics = evaluate_model_quality(
    model,
    tokenizer,
    dataset,
    max_samples=1000,
    batch_size=32
)

print("\nOriginal Model Metrics:")
for key, value in original_metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

Evaluating original model...


Evaluating: 100%|█████████████████████████████████████████████████████| 32/32 [05:16<00:00,  9.90s/it]

[0 1]

Original Model Metrics:
  accuracy: 0.8810
  total_samples: 1000
  correct_predictions: 881
  precision: 0.8917
  recall: 0.8607
  f1_score: 0.8759


In [29]:
print("Evaluating quantized model...")
quantized_metrics = evaluate_model_quality(
    quantized_model,
    tokenizer,
    dataset,
    max_samples=1000,
    batch_size=32
)

print("\nQuantized Model Metrics:")
for key, value in quantized_metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

Evaluating quantized model...


Evaluating: 100%|█████████████████████████████████████████████████████| 32/32 [03:31<00:00,  6.61s/it]

[0 1]

Quantized Model Metrics:
  accuracy: 0.8620
  total_samples: 1000
  correct_predictions: 862
  precision: 0.8692
  recall: 0.8443
  f1_score: 0.8565


In [30]:
accuracy_drop = (original_metrics['accuracy'] - quantized_metrics['accuracy']) * 100
print(f"\nAccuracy drop: {accuracy_drop:.2f}%")

if 'f1_score' in original_metrics:
    f1_drop = (original_metrics['f1_score'] - quantized_metrics['f1_score']) * 100
    print(f"F1 Score drop: {f1_drop:.2f}%")


Accuracy drop: 1.90%
F1 Score drop: 1.94%


### 6️⃣ (Дополнительно) Проверка скорости инференса

In [10]:
print("Measuring inference time for original model...")
original_time = measure_inference_time(
    model,
    tokenizer,
    texts,
    num_runs=100
)

print("\n⚡ Original Model Inference Time:")
print(f"  Mean: {original_time['mean_time']*1000:.2f} ms")
print(f"  Std:  {original_time['std_time']*1000:.2f} ms")

Measuring inference time for original model...


Measuring inference time: 100%|█████████████████████████████████████| 100/100 [00:04<00:00, 20.12it/s]


⚡ Original Model Inference Time:
  Mean: 15.93 ms
  Std:  0.54 ms


In [11]:
print("Measuring inference time for quantized model...")
quantized_time = measure_inference_time(
    quantized_model,
    tokenizer,
    texts,
    num_runs=100
)

print("\n⚡ Quantized Model Inference Time:")
print(f"  Mean: {quantized_time['mean_time']*1000:.2f} ms")
print(f"  Std:  {quantized_time['std_time']*1000:.2f} ms")

Measuring inference time for quantized model...


Measuring inference time: 100%|█████████████████████████████████████| 100/100 [00:02<00:00, 34.41it/s]


⚡ Quantized Model Inference Time:
  Mean: 9.24 ms
  Std:  2.47 ms


In [12]:
speedup = original_time['mean_time'] / quantized_time['mean_time']
print(f"\n🚀 Speedup: {speedup:.2f}x")


🚀 Speedup: 1.72x


### Проверка работы квантованной модели

In [13]:
clf = pipeline("sentiment-analysis", model=quantized_model, tokenizer=tokenizer)

for t in texts:
    print(f"\nText: {t}")
    print("Prediction:", clf(t))

Device set to use cpu



Text: I loved this movie, it was fantastic!
Prediction: [{'label': 'POSITIVE', 'score': 0.9998642206192017}]

Text: This was a terrible waste of time.
Prediction: [{'label': 'NEGATIVE', 'score': 0.9997791647911072}]

Text: Not bad, but could be better.
Prediction: [{'label': 'NEGATIVE', 'score': 0.9467726945877075}]


### Выводы

После применения post-training dynamic quantization:

1. **Размер модели** уменьшился примерно в 2 раза (с ~268 MB до ~138 MB)
2. **Accurcay**: Потеря точности 1.9%
3. **Ускорение** инференса на CPU - 1.72x
